In [33]:
import pandas as pd
import numpy as np
import os, glob, tqdm
from Bio import AlignIO, SeqIO
from scipy.spatial import distance
from matplotlib import rc
from tqdm.notebook import trange, tqdm

%matplotlib inline

# The following %config line changes the inline figures to have a higher DPI.
# You can comment out (#) this line if you don't have a high-DPI (~220) display.
%config InlineBackend.figure_format = 'retina'
# Set the global font to be DejaVu Sans, size 10 (or any other sans-serif font of your choice!)
rc('font',**{'family':'sans-serif','sans-serif':['DejaVu Sans'],'size':12})
coverage = {'02X':0.2, '05X':0.5, '07X':0.5,'1X':1, '5X':5, '10X':10, '20X':20}
tech = {'ont':'ONT', 'nan': 'ONT', 'pac':'PacBIO', 'ill': 'Illumina', np.nan:'ONT'}
spe = {'arath': 'A. thaliana', 'yeast':'S. cerevisiae', 'mouse':'M. musculus'}

# df = pd.read_csv('/Users/daviddylus/projects/r2t/benchmark/r2t_align/r2t+assem_concat_alignments_blast.csv')
# df.head()
moused = pd.read_csv("/Users/daviddylus/Projects/r2t/benchmark/blast/seq_bins/mouse_sp_removal_bin2.csv")
moused['species'] = 'M. musculus'
yeastd = pd.read_csv("/Users/daviddylus/Projects/r2t/benchmark/blast/seq_bins/yeast_sp_removal_bin2.csv")
yeastd['species'] = 'S. cerevisiae'
arathd = pd.read_csv("/Users/daviddylus/Projects/r2t/benchmark/blast/seq_bins/arath_sp_removal_bin2.csv")
arathd['species'] = 'A. thaliana'
datad = pd.concat([moused, yeastd,arathd])
datad = datad.reset_index()

In [34]:
# Open the file with read only permit
ogs_like_shen = pd.DataFrame()
for f in glob.glob('/Users/daviddylus/Projects/r2t/benchmark/blast/r2t_like_shen/*all.txt'):
    
    sp = f.split('/')[-1].split('_')[0]
    fopen = open(f, "r")
    lines = fopen.readlines()
    fopen.close()
    dists = [i.split('_')[3] for i in lines]
    techs = [tech[i.split('_')[4][0:3]] for i in lines]
    covs = [coverage[i.split('_')[4][3:]] for i in lines]
    ogs = [i.split('/')[-1].split('.')[0] for i in lines]
    species = [spe[sp]]*len(ogs)
    tmp = pd.DataFrame({'species':species, 'tech':techs, 'cov':covs, 'dist': dists, 'ogs': ogs})
    ogs_like_shen = pd.concat([ogs_like_shen, tmp])
ogs_like_shen

,species,tech,cov,dist,ogs
0,M. musculus,Illumina,0.2,0,OG10358
1,M. musculus,Illumina,0.2,0,OG10375
2,M. musculus,Illumina,0.2,0,OG10428
3,M. musculus,Illumina,0.2,0,OG11229
4,M. musculus,Illumina,0.2,0,OG11810
...,...,...,...,...,...
2520,S. cerevisiae,PacBIO,5.0,3,OG768685
2521,S. cerevisiae,PacBIO,5.0,3,OG840319
2522,S. cerevisiae,PacBIO,5.0,3,OG762765
2523,S. cerevisiae,PacBIO,5.0,3,OG555588


In [35]:
set(ogs_like_shen['species'])

{'A. thaliana', 'M. musculus', 'S. cerevisiae'}

In [36]:
ogs_like_shen['ogs'][(ogs_like_shen['species']=='M. musculus') & (ogs_like_shen['dist']=='0') & (ogs_like_shen['cov']==0.2) & (ogs_like_shen['tech']=='PacBIO')]

0     OG10209
1     OG10261
2     OG10534
3     OG11144
4     OG11229
5     OG11909
6     OG12369
7     OG12433
8     OG12516
9     OG12947
10    OG13435
11    OG14025
12     OG1406
13    OG15601
14    OG16160
15     OG1713
16    OG17603
17    OG17618
18    OG18409
19     OG1882
20    OG19161
21     OG2058
22    OG22012
23    OG24754
24     OG2493
25    OG26024
26    OG26268
27     OG2778
28     OG2870
29     OG2997
30     OG3270
31     OG3724
32     OG4492
33     OG4624
34     OG4771
35     OG6477
36     OG6631
37     OG7290
38     OG7666
39     OG8298
40     OG8451
41     OG8703
42     OG9048
43     OG9861
Name: ogs, dtype: object

# R2T individually

In [37]:
keep, drop = [], []
for i,r in tqdm(datad.iterrows()):
    sub_ogs = list(ogs_like_shen['ogs'][(ogs_like_shen['species']==r['species']) 
                                        & (ogs_like_shen['dist']==str(r['inode_dist'])) 
                                        & (ogs_like_shen['cov']==r['gcov']) 
                                        & (ogs_like_shen['tech']==r['meth'])])
    
    if r['og'] in sub_ogs:
        keep.append(i)
    else:
        drop.append(i)
#     if 'S. cerevisiae' in r['species']:        
#         print(sub_ogs, r['og'])
#         break

0it [00:00, ?it/s]

In [40]:
datad.iloc[keep].to_csv('/Users/daviddylus/Projects/r2t/benchmark/blast/r2t_like_shen/r2t_blast_like_shen_individuals_from_original.csv', index=False)

In [42]:
data_like_shen = datad.iloc[keep]

In [ ]:
out = {'species':[], 'idist': [], 'tech': [], 'coverage':[], 'evalue':[], 'evalue_stdev':[], 'pident':[], 'scale_pident':[], 
        'pident_stdev':[]}

for f in tqdm(data_like_shen.iterrows()):
    f_name = os.path.basename(f).split('.')[0].split('_')
    if os.path.exists(f) and os.path.getsize(f) > 0:
        df = pd.read_csv(f)
        df['og'] = [r['qseqid'].split('_')[-1] for i,r in df.iterrows()]
        og_list = list(ogs_like_shen[(ogs_like_shen['species']==spe[f_name[0]]) & (ogs_like_shen['dist']==f_name[1]) & (ogs_like_shen['cov']==coverage[f_name[2][3:]]) & (ogs_like_shen['tech']==tech[f_name[2][0:3]])]['ogs'])
        df = df[~df['og'].isin(og_list)]
        pident, scale_pident, evalue = [], [], []
        for query in set(df.qseqid):
            df_sub = df.loc[df.qseqid==query]
            df_sub = df_sub.loc[df_sub['pident'].idxmax()]
            pident.append(df_sub.pident)
            scale_pident.append(df_sub.pident*(df_sub.length/df_sub.qlen))
            evalue.append(df_sub.evalue)

        out['species'].append(spe[f_name[0]])
        out['idist'].append(f_name[1])
        out['tech'].append(tech[f_name[2][0:3]])
        out['coverage'].append(coverage[f_name[2][3:]])
        out['evalue'].append(np.mean(evalue))
        out['evalue_stdev'].append(np.std(evalue))
        out['pident'].append(np.mean(pident))
        out['pident_stdev'].append(np.std(pident))
        out['scale_pident'].append(np.mean(scale_pident))
    else:
        out['species'].append(spe[f_name[0]])
        out['idist'].append(f_name[1])
        out['tech'].append(tech[f_name[2][0:3]])
        out['coverage'].append(coverage[f_name[2][3:]])
        out['evalue'].append(np.nan)
        out['evalue_stdev'].append(np.nan)
        out['pident'].append(np.nan)
        out['pident_stdev'].append(np.nan)
        out['scale_pident'].append(np.nan)